##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 제목

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/io/tutorials/bigtable"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/io/blob/master/docs/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/io/docs/tutorials/bigtable.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

## 개요

이 노트북은 `tensorflow_io.bigtable` 모듈의 기본 사용과 기능을 보여줍니다. 시작하기 전에 이러한 다음과 같은 주제에 익숙한지 확인합니다.

1. [GCP 프로젝트 생성.](https://console.cloud.google.com/cloud-resource-manager)
2. [Bigtable용 Cloud SDK 설](https://cloud.google.com/bigtable/docs/installing-cloud-sdk)
3. [cbt 도구 개요](https://cloud.google.com/bigtable/docs/cbt-overview)
4. [에뮬레이터 사용](https://cloud.google.com/bigtable/docs/emulator)

참고: Jupyter는 앞에 <code>!</code>가 붙은 줄을 셸 명령으로 실행하고 앞에 `$`가 붙은 Python 변수를 이러한 명령에 보간하여 넣습니다.

## 설치

In [ ]:
!pip install tensorflow-io

**참고**: 아래 셀을 실행할 경우 Google Cloud에로그인하라는 요청을 받게 됩니다.

In [ ]:
!mkdir /tools/google-cloud-sdk/.install
!gcloud --quiet components install beta cbt bigtable
!gcloud init

이 예시를 위해, bigtable 에뮬레이터가 사용됩니다. 인스턴스를 설정하고 값을 채운 경우 다음 단계는 건너 뛰고 빠른 시작 섹션으로 바로 이동합니다.

백그라운드에서 에뮬레이터를 시작합니다.

In [ ]:
import os
import subprocess
_emulator = subprocess.Popen(['/tools/google-cloud-sdk/bin/gcloud', 'beta', 'emulators', 'bigtable', 'start', '--host-port=127.0.0.1:8086'],
                                      stdout=subprocess.DEVNULL,
                                      stderr=subprocess.DEVNULL, bufsize=0)

테이블 생성

In [ ]:
%env BIGTABLE_EMULATOR_HOST=127.0.0.1:8086
!cbt -project "test-project" -instance "test-instance" createtable t1 families=cf1 splits=row-a,row-h,row-p,row-z
!cbt -project "test-project" -instance "test-instance" ls


값으로 테이블 채우기

In [ ]:
!cbt -project "test-project" -instance "test-instance" set t1 row-a cf1:c1=A
!cbt -project "test-project" -instance "test-instance" set t1 row-b cf1:c1=B
!cbt -project "test-project" -instance "test-instance" set t1 row-c cf1:c1=C
!cbt -project "test-project" -instance "test-instance" set t1 row-d cf1:c1=D
!cbt -project "test-project" -instance "test-instance" set t1 row-e cf1:c1=E
!cbt -project "test-project" -instance "test-instance" set t1 row-f cf1:c1=F
!cbt -project "test-project" -instance "test-instance" set t1 row-g cf1:c1=G
!cbt -project "test-project" -instance "test-instance" set t1 row-h cf1:c1=H
!cbt -project "test-project" -instance "test-instance" set t1 row-i cf1:c1=I
!cbt -project "test-project" -instance "test-instance" set t1 row-j cf1:c1=J
!cbt -project "test-project" -instance "test-instance" set t1 row-k cf1:c1=K
!cbt -project "test-project" -instance "test-instance" set t1 row-l cf1:c1=L
!cbt -project "test-project" -instance "test-instance" set t1 row-m cf1:c1=M
!cbt -project "test-project" -instance "test-instance" set t1 row-n cf1:c1=N
!cbt -project "test-project" -instance "test-instance" set t1 row-o cf1:c1=O
!cbt -project "test-project" -instance "test-instance" set t1 row-p cf1:c1=P
!cbt -project "test-project" -instance "test-instance" set t1 row-q cf1:c1=Q
!cbt -project "test-project" -instance "test-instance" set t1 row-r cf1:c1=R
!cbt -project "test-project" -instance "test-instance" set t1 row-s cf1:c1=S
!cbt -project "test-project" -instance "test-instance" set t1 row-t cf1:c1=T
!cbt -project "test-project" -instance "test-instance" set t1 row-u cf1:c1=U
!cbt -project "test-project" -instance "test-instance" set t1 row-v cf1:c1=V
!cbt -project "test-project" -instance "test-instance" set t1 row-w cf1:c1=W
!cbt -project "test-project" -instance "test-instance" set t1 row-x cf1:c1=X
!cbt -project "test-project" -instance "test-instance" set t1 row-y cf1:c1=Y
!cbt -project "test-project" -instance "test-instance" set t1 row-z cf1:c1=Z

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_io as tfio
import random

random.seed(10)

## 빠른 시작

우선 읽을 클라이언트와 테이블을 생성해야 합니다.

In [ ]:
# If using your bigtable instance replace the project_id, instance_id 
# and the name of the table with suitable values.

client = tfio.bigtable.BigtableClient(project_id="test-project", instance_id="test-instance")
train_table = client.get_table("t1")

좋습니다! 이제 여러분은 테이블에서 데이터를 읽을 tensorflow 데이터세트를 생성할 수 있습니다.

그러려면 읽고자 하는 데이터 유형, `column_family:column_name` 형식의 열 이름 목록 및 읽고자 하는 row_set을 제공해야 합니다.

row_set을 생성하려면 `tensorflow.bigtable.row_set` 및 `tensorflow.bigtable.row_range` 모듈에서 제공된 유틸리티 메서드를 사용합니다. 여기서 모든 열을 포함하는 row_set이 생성됩니다.

bigtable은 값을 넣은 순서가 아닌 사전식 순서대로 값을 읽습니다. 열은 랜덤 rows-key가 제공되어 뒤섞이게 될 겁니다.

In [ ]:
row_set = tfio.bigtable.row_set.from_rows_or_ranges(tfio.bigtable.row_range.infinite())

train_dataset = train_table.read_rows(["cf1:c1"],row_set, output_type=tf.string)

for tensor in train_dataset:
  print(tensor)

다 됐습니다! 축하합니다!

## 병렬 읽기

데이터세트는 Bigtable에서 병렬 읽기를 지원합니다. 그러려면 `parallel_read_rows` 메서드를 사용하고 `num_parallel_calls`를 인수로 지정합니다. 이 메서드가 호출되면 먼저 작업자 기반 SampleRowKey 간에 작업이 분할됩니다.

**참고**: 병렬로 읽을 때 열은 특정 순서로 읽히지 않습니다.

In [ ]:
for tensor in train_table.parallel_read_rows(["cf1:c1"],row_set=row_set, num_parallel_calls=2):
  print(tensor)

## 특정 row_keys 읽기

Bigtable에서 데이터를 읽으려면, 열 세트나 범위 또는 이들의 조합을 지정할 수 있습니다.

`read_rows` 메서드는 RowSet를 제공해야 합니다. 다음과 같이 특정 열 키 또는 RowRange로 RowSet를 구성할 수 있습니다.

In [ ]:
row_range_below_300 = tfio.bigtable.row_range.right_open("row000", "row300")

my_row_set = tfio.bigtable.row_set.from_rows_or_ranges(row_range_below_300, "row585", "row832")
print(my_row_set)

이러한 row_set는 열 범위 `[row000, row300)` 및 열 row585 및 row832를 포함할 수 있습니다.

무한 범위, 빈 범위, 공백 범위 또는 접두사에서 row_set을 생성할 수도 있습니다. 또한 row_range와 교차할 수도 있습니다.

In [ ]:
my_truncated_row_set = tfio.bigtable.row_set.intersect(my_row_set,
                                         tfio.bigtable.row_range.right_open("row200", "row700"))
print(my_truncated_row_set)

## 값 버전 지정

Bigtable을 사용하여 타임스탬프가 다른 하나의 셀에 많은 값을 보관할 수 있습니다. 버전 필터를 사용하여 선택하고자 하는 버전을 지정할 수 있습니다. 하지만, tensorflow.bigtable 커넥터를 사용하여 오직 2차원 벡터만 검색할 수 있어, `latest` 필터가 항상 사용자 지정 버전 필터에 추가됩니다. 즉, 하나의 셀에 대해 2개 이상의 값이 제공된 필터를 통과하는 경우, 새로운 값이 사용됩니다.

최신 값을 전달하는 `latest` 필터를 사용하거나 시간 범위를 지정할 수 있습니다. 시간 범위는 Python datetime 객체 또는 epoch 이후 초 또는 마이크로초를 나타내는 숫자로 제공될 수 있습니다.

In [ ]:
from datetime import datetime

start = datetime(2020, 10, 10, 12, 0, 0)
end = datetime(2100, 10, 10, 13, 0, 0)
from_datetime = tfio.bigtable.filters.timestamp_range(start, end)
from_posix_timestamp = tfio.bigtable.filters.timestamp_range(int(start.timestamp()), int(end.timestamp()))
print("from_datetime:", from_datetime)

print("from_posix_timestamp:", from_posix_timestamp)